In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import datetime
from pyramid.arima import auto_arima

Matplotlib is building the font cache; this may take a moment.


ImportError: cannot import name 'six'

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/alext234/coronavirus-stats/master/data/who-global-cases.csv',index_col=0)

In [ ]:
del data['report_url']

In [ ]:
data = data[~data.index.duplicated(keep='last')]

In [ ]:
data.describe()

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.global_cases.plot(figsize=(15, 5), title='Infected People')
(data.global_cases - data.global_cases.shift(1)).plot(figsize=(15, 5))

In [ ]:
predicted_days = 5
firstPredictionDay = datetime.datetime.strptime(data.index[-1], '%Y-%m-%d') + datetime.timedelta(days = 1)

In [ ]:
stepwise_model = auto_arima(data.global_cases,
                            seasonal=False,
                            trace=True,
                            error_action='ignore',  
                            suppress_warnings=True, 
                            stepwise=True)

In [ ]:
stepwise_model.aic()

In [ ]:
stepwise_model.fit(data.global_cases)

In [ ]:
future_forecast = stepwise_model.predict(n_periods=predicted_days)

In [ ]:
for x in range (0, predicted_days):
    print(str(firstPredictionDay + datetime.timedelta(days = x))[0:10] + " - " +str(int(future_forecast[x])))